In [323]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import warnings
import seaborn as sns
from scipy.stats import probplot,skew
warnings.filterwarnings('ignore')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
# data_desc = pd.read_csv('data_description.txt')
train_ID = train['Id']
test_ID = test['Id']

train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)
train.shape

(1460, 80)

In [324]:
# TODO one 清理异常值
# 通过相关系数分析和求得需求最相关的几个变量
# train.corr()['SalePrice'].sort_values(ascending=False)

In [325]:
# 根据最相关的几个变量清理异常值
# 以下三种结果一样
# 第一个 修改kind 可以变为折线图
# sns.relplot(x = 'GrLivArea',y = 'SalePrice',data=train) 
# sns.regplot(train['GrLivArea'],train['SalePrice'])
# sns.scatterplot(train['GrLivArea'],train['SalePrice'])
train.shape

(1460, 80)

In [326]:
# 对数据进行整合。 分开特征和标签
# 剔除异常值
train = train.drop(train[(train['GrLivArea']>4000)&(train['SalePrice']<300000)].index)
ntrain = train.shape[0]
ntest = test.shape[0]
train['SalePrice'] = np.log1p(train['SalePrice'])
y_train = train.SalePrice.values # 区分特征和标签
all_data = pd.concat((train,test)).reset_index(drop=True)
all_data.drop(['SalePrice'],axis=1,inplace=True)
# all_data.shape
y_train


array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

In [327]:
# TODO 分析预测变量
"""
sns.distplot :可视化变量分布
sns.scatterplot/relplot/:可视化统计关系/可散点
sns.catplot:用分类数据绘图/散点/
sns.regplot: 线性
"""
# sns.distplot(train['SalePrice']) # 通过图可以得知，数据偏度较大，需要对数据进行处理 使其正态化，线性化
# sns.distplot(np.log10(train['SalePrice']),bins=300,rug=True)
# fig,ax = plt.subplots()
# stats.probplot(np.log2(train['SalePrice']),plot=plt)

'\nsns.distplot :可视化变量分布\nsns.scatterplot/relplot/:可视化统计关系/可散点\nsns.catplot:用分类数据绘图/散点/\nsns.regplot: 线性\n'

In [328]:
# # 空值占比排序
all_data_na = ((all_data.isnull().sum()/len(all_data))*100)
all_data_na = all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
all_data_na.head(20)

PoolQC          99.691464
MiscFeature     96.400411
Alley           93.212204
Fence           80.425094
FireplaceQu     48.680151
LotFrontage     16.660953
GarageQual       5.450806
GarageCond       5.450806
GarageFinish     5.450806
GarageYrBlt      5.450806
GarageType       5.382242
BsmtExposure     2.811107
BsmtCond         2.811107
BsmtQual         2.776826
BsmtFinType2     2.742544
BsmtFinType1     2.708262
MasVnrType       0.822763
MasVnrArea       0.788481
MSZoning         0.137127
BsmtFullBath     0.068564
dtype: float64

In [329]:
# fig,ax =plt.subplots(figsize=(7,4)) # 在sns中构造plt的话，始对sns一些属性进行修改
# sns.barplot(all_data_na.index,all_data_na)
# plt.xticks(rotation=90)
# # plt.yticks()

In [330]:
# 查看一些属性之间是否包含线性关系
# all_data.corr()
# sns.heatmap(all_data.corr())

In [331]:
# 开始对空值进行处理
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
# all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)
all_data = all_data.drop(['Utilities'], axis=1)

all_data.shape

(2917, 78)

In [332]:
# 对离散型变量进行编码 编为123456
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond',
'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1',
'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond',
'YrSold', 'MoSold')# process columns, apply LabelEncoder to categorical features
all_data['FireplaceQu']
for c in cols:
    lbl = LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c] = lbl.transform(list(all_data[c].values))
# all_data['FireplaceQu']
all_data.shape

(2917, 78)

In [333]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
all_data.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,TotalSF
0,856,854,0,1,3,1Fam,4,3,706.0,0.0,...,WD,0,1,8,856.0,0,2003,2003,2,2566.0
1,1262,0,0,1,3,1Fam,4,1,978.0,0.0,...,WD,0,1,6,1262.0,298,1976,1976,1,2524.0
2,920,866,0,1,3,1Fam,4,2,486.0,0.0,...,WD,0,1,6,920.0,0,2001,2002,2,2706.0
3,961,756,0,1,3,1Fam,1,3,216.0,0.0,...,WD,0,1,7,756.0,0,1915,1970,0,2473.0
4,1145,1053,0,1,4,1Fam,4,0,655.0,0.0,...,WD,0,1,9,1145.0,192,2000,2000,2,3343.0


In [334]:
# 1. 把数值类型的数据找出来，然后计算偏度
numeric_feats  = all_data.dtypes[all_data.dtypes!='object'].index # all_data.dtype返回的是字典【】all_data.dtypes!='object'返回的是字典，值是布尔
skew_feat = all_data[numeric_feats].apply(lambda x:skew(x.dropna())).sort_values(ascending=False) # 删除空值，同时计算偏度并应用
skewness = pd.DataFrame({'Skew' :skew_feat})
skewness.head(10)

,Skew
MiscVal,21.939672
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080
LandSlope,4.973254
KitchenAbvGr,4.300550
BsmtFinSF2,4.144503
EnclosedPorch,4.002344
ScreenPorch,3.945101


In [335]:
# 对具有偏度的特征进行数据结构转换 log 或者box
"""
boxcox
是根据输如的连续变量，然后变换参数拉姆塔，参数=0/对数，参数=-1/倒数，参数=0.5/平方根
参数的估计方法：最大似然/贝叶斯"""
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
#all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)
all_data.head()

There are 59 skewed numerical features to Box Cox transform


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,TotalSF
0,11.692623,11.686189,0.0,0.730463,1.540963,1Fam,1.820334,1.540963,11.170327,0.0,...,WD,0.0,0.730463,2.602594,11.692623,0.000000,14.187527,14.187527,1.194318,14.976591
1,12.792276,0.000000,0.0,0.730463,1.540963,1Fam,1.820334,0.730463,12.062832,0.0,...,WD,0.0,0.730463,2.259674,12.792276,9.010206,14.145138,14.145138,0.730463,14.923100
2,11.892039,11.724598,0.0,0.730463,1.540963,1Fam,1.820334,1.194318,10.200343,0.0,...,WD,0.0,0.730463,2.259674,11.892039,0.000000,14.184404,14.185966,1.194318,15.149678
3,12.013683,11.354094,0.0,0.730463,1.540963,1Fam,0.730463,1.540963,8.274266,0.0,...,WD,0.0,0.730463,2.440268,11.354094,0.000000,14.047529,14.135652,0.000000,14.857121
4,12.510588,12.271365,0.0,0.730463,1.820334,1Fam,1.820334,0.000000,10.971129,0.0,...,WD,0.0,0.730463,2.750250,12.510588,8.013884,14.182841,14.182841,1.194318,15.852312


In [336]:
all_data = pd.get_dummies(all_data) # 实现one-hot-encode
all_data.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,11.692623,11.686189,0.0,0.730463,1.540963,1.820334,1.540963,11.170327,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1
1,12.792276,0.000000,0.0,0.730463,1.540963,1.820334,0.730463,12.062832,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,11.892039,11.724598,0.0,0.730463,1.540963,1.820334,1.194318,10.200343,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1
3,12.013683,11.354094,0.0,0.730463,1.540963,0.730463,1.540963,8.274266,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
4,12.510588,12.271365,0.0,0.730463,1.820334,1.820334,0.000000,10.971129,0.0,1.194318,...,0,0,0,0,0,0,0,0,0,1


In [337]:
train = all_data[:ntrain] # 训练特征
test = all_data[ntrain:] # 测试特征
# print(train.shape)
# print(test.shape)
# print(train.head())
# print(test.head())
# print(train.values)
# print(ntrain)
all_data

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,11.692623,11.686189,0.000000,0.730463,1.540963,1.820334,1.540963,11.170327,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1
1,12.792276,0.000000,0.000000,0.730463,1.540963,1.820334,0.730463,12.062832,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,11.892039,11.724598,0.000000,0.730463,1.540963,1.820334,1.194318,10.200343,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1
3,12.013683,11.354094,0.000000,0.730463,1.540963,0.730463,1.540963,8.274266,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
4,12.510588,12.271365,0.000000,0.730463,1.820334,1.820334,0.000000,10.971129,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1
5,11.493820,10.589574,9.178051,0.730463,0.730463,1.820334,1.540963,11.267217,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1
6,13.670207,0.000000,0.000000,0.730463,1.540963,1.820334,0.000000,13.031093,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1
7,12.413831,12.077149,0.000000,0.730463,1.540963,1.820334,1.194318,11.702249,4.597157,0.000000,...,0,0,0,0,0,0,0,0,0,1
8,12.186751,11.339779,0.000000,0.730463,1.194318,1.820334,1.540963,0.000000,0.000000,2.259674,...,0,0,0,0,0,0,0,0,0,1
9,12.335431,0.000000,0.000000,0.730463,1.194318,1.820334,1.540963,11.676516,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1


In [338]:
# 模型
"""                               弹性网络   套索  岭回归 贝叶斯岭回归  联合套索"""
from sklearn.linear_model import ElasticNet,Lasso,Ridge,BayesianRidge,LassoLarsIC
#如果你的数据包含许多异常值，使用均值和方差缩放可能并不是一个很好的选择。
#这种情况下，你可以使用 robust_scale 以及 RobustScaler 作为替代品。它们对你的数据的中心和范围使用更有鲁棒性的估计。
from sklearn.preprocessing import RobustScaler#
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import KFold,train_test_split,cross_val_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from sklearn.metrics import mean_squared_error

In [339]:
# 构建框架
# 构建交叉函数
#Validation functionn_folds = 5
n_folds=5
print(train.values)
print(y_train)
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

[[11.69262259 11.68618938  0.         ...  0.          0.
   1.        ]
 [12.79227623  0.          0.         ...  0.          0.
   1.        ]
 [11.89203906 11.72459848  0.         ...  0.          0.
   1.        ]
 ...
 [12.6168403  12.52811325  0.         ...  0.          0.
   1.        ]
 [12.33807443  0.          0.         ...  0.          0.
   1.        ]
 [12.77838192  0.          0.         ...  0.          0.
   1.        ]]
[12.24769912 12.10901644 12.31717117 ... 12.49313327 11.86446927
 11.90159023]


In [340]:
# krr =KernelRidge(alpha=0.5,kernel='polynomial',degree=2,coef0=2.5)
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha =0.0005, random_state=3,l1_ratio=0.9))
KRR = make_pipeline(RobustScaler(), KernelRidge(alpha =0.6,kernel='polynomial',degree=2,coef0=2.5))
GBoost = GradientBoostingRegressor(n_estimators=3000,learning_rate=0.05,
                                  max_depth=4,max_features='sqrt',
                                  min_samples_leaf=15,min_samples_split=10,
                                  loss='huber',random_state=5)
model_xgb = xgb.XGBRegressor(colsample_bylevel=0.4603,gamma=0.0468,
                            learning_rate=0.05, max_depth=3,
min_child_weight=1.7817, n_estimators=2200,
reg_alpha=0.4640, reg_lambda=0.8571,
subsample=0.5213, silent=1,
random_state =7, nthread = -1)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
learning_rate=0.05, n_estimators=720,
max_bin = 55, bagging_fraction = 0.8,
bagging_freq = 5, feature_fraction = 0.2319,
feature_fraction_seed=9, bagging_seed=9,
min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [341]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))



Lasso score: 0.1115 (0.0074)

ElasticNet score: 0.1116 (0.0074)

Kernel Ridge score: 0.1180 (0.0090)

Gradient Boosting score: 0.1177 (0.0080)

Xgboost score: 0.1163 (0.0057)

LGBM score: 0.1161 (0.0067)



In [342]:
# 计算平均分数
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
# we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
# Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
#Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
        model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [352]:
# 堆叠模型
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        print(self.base_models_)
        self.meta_model_ = clone(self.meta_model)
#         print(self.meta_model_)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
    
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        # 一共有样本数量行，每行有三列 均为0
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                print(train_index)
                # train_index 特征索引 holdout_index 标签索引
                instance = clone(model)
                self.base_models_[i].append(instance)
                # 把每个基模型，对数据进行交叉验证
                instance.fit(X[train_index], y[train_index])
                # 基模型获得预测结果
                y_pred = instance.predict(X[holdout_index])
                # 把预测结果添加到相应测试模型的位置，把基模型预测的结果作为特征
                out_of_fold_predictions[holdout_index, i] = y_pred
                # Now train the cloned meta-model using the out-of-fold predictions as new feature
        # 基模型作为特征，元模型作为预测模型
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
        #Do the predictions of all base models on the test data and use the averaged predictions as
        #meta-features for the final prediction which is done by the meta-model
    def predict(self, X): #np.column_stack 对数据进行叠加
        meta_features = np.column_stack([
        np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
        for base_models in self.base_models_])
        return self.meta_model_.predict(meta_features)
# StackingAveragedModels(base_models = (ENet, GBoost, KRR),
# meta_model = lasso)
# print('---------------------------------------------------------')

# stacked_averaged_models.fit(train.values,y_train)
# stacked_train_pred = stacked_averaged_models.predict(train.values)
# stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
# print(rmsle(y_train, stacked_train_pred))


In [314]:
# 使用弹性网络，核函数，gboost作为基模型，套索作为元模型
# stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
# meta_model = lasso)
# score = rmsle_cv(stacked_averaged_models)
# print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [298]:
def rmsle(y,y_pred):
    return np.sqrt(mean_squared_error(y,y_pred))

In [301]:
# 堆叠集成np.expm1 log1平滑处理后的逆运算
stacked_averaged_models.fit(train.values,y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

0.07791545487165992


In [303]:
# XGB 集成 np.expm1 log1平滑处理后的逆运算
model_xgb.fit(train,y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

0.07829124722858676


In [304]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

0.07267779757894059


In [305]:
print(rmsle(y_train,stacked_train_pred*0.70 +
xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

0.07516783338701317


In [306]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [308]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])